# 分子数据转换为图结构

## 1. 数据提取

In [1]:
# 读取 QM9_demo 数据集
from utils.data import QM9DataExtractor

# 使用定义的QM9DataExtractor类用于加载提取数据
extractor = QM9DataExtractor('../data/demo/qm9_demo.csv')

**测试数据提取功能**

In [2]:
# 使用 SMILES 字符串 "C"（甲烷）测试数据提取功能
info = extractor.get_molecule_info('C')
print("数据结构:")
print("坐标数据keys:", info['coordinates_data'].keys())
print("属性数据keys:", info['properties_data'].keys())
info

数据结构:
坐标数据keys: dict_keys(['smiles', 'atoms', 'coordinates', 'charges', 'num_atoms'])
属性数据keys: dict_keys(['smiles', 'properties', 'property_vector'])


{'coordinates_data': {'smiles': 'C',
  'atoms': ['C', 'H', 'H', 'H', 'H'],
  'coordinates': array([[-1.26981359e-02,  1.08580416e+00,  8.00099580e-03],
         [ 2.15041600e-03, -6.03131760e-03,  1.97612040e-03],
         [ 1.01173084e+00,  1.46375116e+00,  2.76574800e-04],
         [-5.40815069e-01,  1.44752661e+00, -8.76643715e-01],
         [-5.23813634e-01,  1.43793264e+00,  9.06397294e-01]]),
  'charges': array([-0.535689,  0.133921,  0.133922,  0.133923,  0.133923]),
  'num_atoms': 5},
 'properties_data': {'smiles': 'C',
  'properties': {'A': 157.7118,
   'B': 157.70997,
   'C': 157.70699,
   'miu': 0.0,
   'alpha': 13.21,
   'homo': -0.3877,
   'lumo': 0.1171,
   'gap': 0.5048,
   'R2': 35.3641,
   'zpve': 0.044749,
   'Uo': -40.47893,
   'U': -40.476062,
   'H': -40.475117,
   'G': -40.498597,
   'Cv': 6.469},
  'property_vector': array([ 1.5771180e+02,  1.5770997e+02,  1.5770699e+02,  0.0000000e+00,
          1.3210000e+01, -3.8770000e-01,  1.1710000e-01,  5.0480000e-01,
    

## 2. 图结构转换

**节点、边特征定义**
- 节点：
1. 原子类型（one-hot编码, H, C, N, O, F）
2. 原子坐标（x, y, z）
3. 原子电荷

- 边：
1. 几何距离（键长）
2. 化学键类型（one-hot编码, 单键、双键、三键、芳香键等）

In [3]:
import numpy as np

# 从重构后的数据结构中提取坐标数据
coords_data = info['coordinates_data']
atoms = coords_data['atoms']
coordinates = coords_data['coordinates']
charges = coords_data['charges']

atom_types = ['H', 'C', 'N', 'O', 'F']  # QM9中常见的原子类型
node_features = []

for i, atom in enumerate(atoms):
    # 原子类型 one-hot 编码
    atom_feature = [0] * len(atom_types)
    if atom in atom_types:
        atom_feature[atom_types.index(atom)] = 1

    # 添加原子电荷作为特征
    atom_feature.append(charges[i])

    # 添加坐标作为特征
    atom_feature.extend(coordinates[i].tolist())

    node_features.append(atom_feature)

node_features = np.array(node_features, dtype=np.float32)

print(f"节点特征矩阵形状: {node_features.shape}")
print(f"每个节点特征维度: {node_features.shape[1]} (原子类型5维 + 电荷1维 + 坐标3维)")
node_features

节点特征矩阵形状: (5, 9)
每个节点特征维度: 9 (原子类型5维 + 电荷1维 + 坐标3维)


array([[ 0.0000000e+00,  1.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00, -5.3568900e-01, -1.2698136e-02,  1.0858041e+00,
         8.0009960e-03],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  1.3392100e-01,  2.1504159e-03, -6.0313176e-03,
         1.9761203e-03],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  1.3392200e-01,  1.0117308e+00,  1.4637512e+00,
         2.7657481e-04],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  1.3392299e-01, -5.4081506e-01,  1.4475266e+00,
        -8.7664372e-01],
       [ 1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
         0.0000000e+00,  1.3392299e-01, -5.2381361e-01,  1.4379326e+00,
         9.0639728e-01]], dtype=float32)

In [4]:
# 构建边特征（基于距离阈值）
distance_threshold = 1.8  # Angstrom阈值，可调整
edges = []
edge_features = []
num_atoms = len(atoms)

print(f"构建图的边连接，原子数量: {num_atoms}")

# 计算所有原子对之间的距离
for i in range(num_atoms):
    for j in range(i + 1, num_atoms):
        # 计算欧几里得距离
        dist = np.linalg.norm(coordinates[i] - coordinates[j])

        # 如果距离小于阈值，创建边
        if dist < distance_threshold:
            print(f"原子 {i}({atoms[i]}) - 原子 {j}({atoms[j]}): 距离 {dist:.3f} Å")

            # 添加双向边（无向图）
            edges.append([i, j])
            edges.append([j, i])

            # 边特征：距离和距离的倒数
            edge_feat = [dist, 1.0 / dist]
            edge_features.append(edge_feat)
            edge_features.append(edge_feat)  # 双向边使用相同特征

# 转换为numpy数组
if edges:
    edges = np.array(edges, dtype=np.int64)
    edge_features = np.array(edge_features, dtype=np.float32)
    print(f"边索引矩阵形状: {edges.shape}")
    print(f"边特征矩阵形状: {edge_features.shape}")
else:
    edges = np.array([], dtype=np.int64).reshape(0, 2)
    edge_features = np.array([], dtype=np.float32).reshape(0, 2)
    print("未发现符合阈值的边连接")

print(f"图结构: {num_atoms}个节点, {len(edges)}条边")

构建图的边连接，原子数量: 5
原子 0(C) - 原子 1(H): 距离 1.092 Å
原子 0(C) - 原子 2(H): 距离 1.092 Å
原子 0(C) - 原子 3(H): 距离 1.092 Å
原子 0(C) - 原子 4(H): 距离 1.092 Å
原子 1(H) - 原子 2(H): 距离 1.783 Å
原子 1(H) - 原子 3(H): 距离 1.783 Å
原子 1(H) - 原子 4(H): 距离 1.783 Å
原子 2(H) - 原子 3(H): 距离 1.783 Å
原子 2(H) - 原子 4(H): 距离 1.783 Å
原子 3(H) - 原子 4(H): 距离 1.783 Å
边索引矩阵形状: (20, 2)
边特征矩阵形状: (20, 2)
图结构: 5个节点, 20条边


In [5]:
# 可视化图结构信息
print("=== 图结构总结 ===")
print(f"分子SMILES: {coords_data['smiles']}")
print(f"原子类型: {atoms}")
print(f"节点特征: {node_features.shape}")
print(f"边连接: {edges.shape if len(edges) > 0 else '无边'}")
print(f"边特征: {edge_features.shape if len(edge_features) > 0 else '无特征'}")

# 展示边连接详情
if len(edges) > 0:
    print("\n边连接详情:")
    unique_edges = edges[::2]  # 去除重复的双向边，只显示一次
    for idx, (i, j) in enumerate(unique_edges):
        dist = edge_features[idx * 2][0]  # 获取距离值
        print(f"  {atoms[i]}{i} -- {atoms[j]}{j}: {dist:.3f} Å")


=== 图结构总结 ===
分子SMILES: C
原子类型: ['C', 'H', 'H', 'H', 'H']
节点特征: (5, 9)
边连接: (20, 2)
边特征: (20, 2)

边连接详情:
  C0 -- H1: 1.092 Å
  C0 -- H2: 1.092 Å
  C0 -- H3: 1.092 Å
  C0 -- H4: 1.092 Å
  H1 -- H2: 1.783 Å
  H1 -- H3: 1.783 Å
  H1 -- H4: 1.783 Å
  H2 -- H3: 1.783 Å
  H2 -- H4: 1.783 Å
  H3 -- H4: 1.783 Å


In [6]:
# 根据 SMILES 获得键类型
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

def get_bond_features_from_smiles(smiles, atoms, coordinates, distance_threshold=1.8):
    """
    根据SMILES字符串获取化学键特征

    Args:
        smiles: SMILES字符串
        atoms: 原子列表
        coordinates: 原子坐标
        distance_threshold: 距离阈值

    Returns:
        edges: 边索引列表
        edge_features: 增强的边特征(距离 + 键类型)
    """
    # 解析SMILES分子并添加氢原子
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"无法解析SMILES: {smiles}")
        return [], []

    # 显式添加氢原子
    mol = Chem.AddHs(mol)

    # 键类型映射
    bond_type_map = {
        Chem.BondType.SINGLE: [1, 0, 0, 0],      # 单键
        Chem.BondType.DOUBLE: [0, 1, 0, 0],      # 双键
        Chem.BondType.TRIPLE: [0, 0, 1, 0],      # 三键
        Chem.BondType.AROMATIC: [0, 0, 0, 1]     # 芳香键
    }

    edges = []
    edge_features = []
    num_atoms = len(atoms)

    print(f"SMILES分子包含 {mol.GetNumAtoms()} 个原子, {mol.GetNumBonds()} 个化学键")
    print(f"坐标数据包含 {num_atoms} 个原子")

    # 检查原子数量是否匹配
    if mol.GetNumAtoms() != num_atoms:
        print(f"警告: SMILES原子数({mol.GetNumAtoms()})与坐标原子数({num_atoms})不匹配")
        # 尝试不同的策略或返回基于距离的边

    # 首先构建基于距离的边连接
    distance_edges = {}
    for i in range(num_atoms):
        for j in range(i + 1, num_atoms):
            try:
                # 安全的距离计算，处理科学计数法
                coord_i = np.array(coordinates[i], dtype=np.float64)
                coord_j = np.array(coordinates[j], dtype=np.float64)
                dist = float(np.linalg.norm(coord_i - coord_j))

                if dist < distance_threshold:
                    distance_edges[(i, j)] = dist
            except (ValueError, TypeError) as e:
                print(f"坐标转换错误: 原子{i}-{j}, 错误: {e}")
                continue

    # 获取SMILES中的化学键信息
    smiles_bonds = {}
    for bond in mol.GetBonds():
        begin_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        bond_type = bond.GetBondType()

        # 确保索引顺序一致
        edge_key = (min(begin_idx, end_idx), max(begin_idx, end_idx))
        smiles_bonds[edge_key] = bond_type

        # 检查索引是否在有效范围内
        if begin_idx < len(atoms) and end_idx < len(atoms):
            print(f"SMILES键: 原子{begin_idx}({atoms[begin_idx]}) - "
                  f"原子{end_idx}({atoms[end_idx]}): {bond_type}")
        else:
            print(f"SMILES键: 原子{begin_idx} - 原子{end_idx}: {bond_type} (索引超出范围)")

    # 合并距离边和SMILES键信息
    for (i, j), dist in distance_edges.items():
        try:
            # 基础边特征: [距离, 距离倒数] - 确保数值安全转换
            base_features = [float(dist), float(1.0 / dist) if dist > 0 else 0.0]

            # 化学键类型特征
            edge_key = (i, j)
            if edge_key in smiles_bonds:
                # 有SMILES键信息
                bond_type = smiles_bonds[edge_key]
                bond_features = bond_type_map.get(bond_type, [0, 0, 0, 0])
                print(f"匹配键: 原子{i}({atoms[i]}) - 原子{j}({atoms[j]}): "
                      f"距离{dist:.3f}Å, 类型{bond_type}")
            else:
                # 无SMILES键信息，标记为非键连接
                bond_features = [0, 0, 0, 0]
                print(f"非键连接: 原子{i}({atoms[i]}) - 原子{j}({atoms[j]}): 距离{dist:.3f}Å")

            # 组合特征: [距离, 距离倒数, 单键, 双键, 三键, 芳香键]
            edge_feat = base_features + bond_features

            # 添加双向边
            edges.extend([[i, j], [j, i]])
            edge_features.extend([edge_feat, edge_feat])

        except (ValueError, TypeError) as e:
            print(f"边特征计算错误: 原子{i}-{j}, 错误: {e}")
            continue

    return edges, edge_features

# 使用新的键特征提取函数
smiles = coords_data['smiles']
enhanced_edges, enhanced_edge_features = get_bond_features_from_smiles(
    smiles, atoms, coordinates, distance_threshold
)

if enhanced_edges:
    enhanced_edges = np.array(enhanced_edges, dtype=np.int64)
    enhanced_edge_features = np.array(enhanced_edge_features, dtype=np.float32)

    print(f"\n=== 增强边特征结果 ===")
    print(f"边索引矩阵形状: {enhanced_edges.shape}")
    print(f"边特征矩阵形状: {enhanced_edge_features.shape}")
    print(f"边特征维度: {enhanced_edge_features.shape[1]} (距离2维 + 键类型4维)")

    # 显示边特征详情
    print(f"\n边特征详情:")
    unique_enhanced_edges = enhanced_edges[::2]
    for idx, (i, j) in enumerate(unique_enhanced_edges):
        feat = enhanced_edge_features[idx * 2]
        dist, inv_dist = feat[0], feat[1]
        bond_type_feat = feat[2:6]

        # 解释键类型
        bond_types = ['单键', '双键', '三键', '芳香键']
        active_bonds = [bond_types[k] for k, v in enumerate(bond_type_feat) if v == 1]
        bond_desc = active_bonds[0] if active_bonds else '非键连接'

        print(f"  {atoms[i]}{i} -- {atoms[j]}{j}: {dist:.3f}Å, {bond_desc}")
else:
    print("未发现任何边连接")


SMILES分子包含 5 个原子, 4 个化学键
坐标数据包含 5 个原子
SMILES键: 原子0(C) - 原子1(H): SINGLE
SMILES键: 原子0(C) - 原子2(H): SINGLE
SMILES键: 原子0(C) - 原子3(H): SINGLE
SMILES键: 原子0(C) - 原子4(H): SINGLE
匹配键: 原子0(C) - 原子1(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子2(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子3(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子4(H): 距离1.092Å, 类型SINGLE
非键连接: 原子1(H) - 原子2(H): 距离1.783Å
非键连接: 原子1(H) - 原子3(H): 距离1.783Å
非键连接: 原子1(H) - 原子4(H): 距离1.783Å
非键连接: 原子2(H) - 原子3(H): 距离1.783Å
非键连接: 原子2(H) - 原子4(H): 距离1.783Å
非键连接: 原子3(H) - 原子4(H): 距离1.783Å

=== 增强边特征结果 ===
边索引矩阵形状: (20, 2)
边特征矩阵形状: (20, 6)
边特征维度: 6 (距离2维 + 键类型4维)

边特征详情:
  C0 -- H1: 1.092Å, 单键
  C0 -- H2: 1.092Å, 单键
  C0 -- H3: 1.092Å, 单键
  C0 -- H4: 1.092Å, 单键
  H1 -- H2: 1.783Å, 非键连接
  H1 -- H3: 1.783Å, 非键连接
  H1 -- H4: 1.783Å, 非键连接
  H2 -- H3: 1.783Å, 非键连接
  H2 -- H4: 1.783Å, 非键连接
  H3 -- H4: 1.783Å, 非键连接


In [7]:
# 保存图结构数据
graph_data = {
    'smiles': smiles,
    'atoms': atoms,
    'num_atoms': len(atoms),
    'node_features': node_features,
    'edges': enhanced_edges if len(enhanced_edges) > 0 else np.array([], dtype=np.int64).reshape(0, 2),
    'edge_features': enhanced_edge_features if len(enhanced_edge_features) > 0 else np.array([], dtype=np.float32).reshape(0, 6),
    'properties': info['properties_data']['properties']
}

print(f"\n=== 最终图结构 ===")
print(f"分子: {smiles}")
print(f"节点数: {graph_data['num_atoms']}")
print(f"边数: {len(graph_data['edges'])}")
print(f"节点特征维度: {graph_data['node_features'].shape[1] if len(graph_data['node_features']) > 0 else 0}")
print(f"边特征维度: {graph_data['edge_features'].shape[1] if len(graph_data['edge_features']) > 0 else 0}")
print(f"分子属性: {len(graph_data['properties'])}个量子化学属性")



=== 最终图结构 ===
分子: C
节点数: 5
边数: 20
节点特征维度: 9
边特征维度: 6
分子属性: 15个量子化学属性


In [8]:
from utils.data import safe_float_convert
# 定义函数封装转换逻辑
from typing import Dict, List, Tuple

def mol_to_graph(mol_data: Dict, distance_threshold=1.8) -> Dict:
    """
    将分子数据转换为图结构

    Args:
        mol_data: 包含分子信息的字典，需包含 'smiles', 'atoms', 'coordinates', 'charges' 等键
        distance_threshold: 用于构建边连接的距离阈值

    Returns:
        graph_data: 包含图结构信息的字典
    """
    atoms = mol_data['atoms']
    coordinates = mol_data['coordinates']
    charges = mol_data['charges']

    atom_types = ['H', 'C', 'N', 'O', 'F']  # QM9中常见的原子类型
    node_features = []

    for i, atom in enumerate(atoms):
        # 原子类型 one-hot 编码
        atom_feature = [0] * len(atom_types)
        if atom in atom_types:
            atom_feature[atom_types.index(atom)] = 1

        # 添加原子电荷作为特征 - 使用安全转换
        atom_feature.append(safe_float_convert(charges[i]))

        # 添加坐标作为特征 - 使用安全转换
        try:
            coord = coordinates[i]
            if hasattr(coord, 'tolist'):
                coord_list = coord.tolist()
            else:
                coord_list = list(coord)
            atom_feature.extend([safe_float_convert(x) for x in coord_list])
        except (ValueError, TypeError):
            atom_feature.extend([0.0, 0.0, 0.0])

        node_features.append(atom_feature)

    node_features = np.array(node_features, dtype=np.float32)
    num_atoms = len(atoms)

    # 构建边和边特征
    edges, edge_features = get_bond_features_from_smiles(
        mol_data['smiles'], atoms, coordinates, distance_threshold
    )

    if edges and len(edges) > 0:
        edges = np.array(edges, dtype=np.int64)
        edge_features = np.array(edge_features, dtype=np.float32)
    else:
        edges = np.array([], dtype=np.int64).reshape(0, 2)
        edge_features = np.array([], dtype=np.float32).reshape(0, 6)

    # 构建邻接矩阵（可选，某些GNN框架需要） - 安全构建
    adjacency_matrix = np.zeros((num_atoms, num_atoms), dtype=np.float32)
    if len(edges) > 0:
        for edge_idx in range(len(edges)):
            i, j = edges[edge_idx]
            if 0 <= i < num_atoms and 0 <= j < num_atoms:
                adjacency_matrix[i, j] = 1.0

    # 打包图数据
    graph_data = {
        'num_nodes': num_atoms,
        'node_features': node_features,  # shape: (num_atoms, feature_dim)
        'edges': edges,                   # shape: (num_edges, 2)
        'edge_features': edge_features,   # shape: (num_edges, edge_feat_dim)
        'adjacency_matrix': adjacency_matrix,
        'smiles': mol_data['smiles'],
        'atoms': atoms,
        'properties': mol_data.get('properties', {})
    }
    return graph_data


# 使用封装函数转换分子为图
graph = mol_to_graph({
    'smiles': coords_data['smiles'],
    'atoms': atoms,
    'coordinates': coordinates,
    'charges': charges,
    'properties': info['properties_data']['properties']
}, distance_threshold)
print(f"\n=== 使用封装函数转换的图结构 ===")
print(f"分子: {graph['smiles']}")
print(f"节点数: {graph['num_nodes']}")
print(f"边数: {len(graph['edges'])}")
print(f"节点特征维度: {graph['node_features'].shape[1]
    if len(graph['node_features']) > 0 else 0}")
print(f"边特征维度: {graph['edge_features'].shape[1]
    if len(graph['edge_features']) > 0 else 0}")
print(f"分子属性: {len(graph['properties'])}个量子化学属性")

SMILES分子包含 5 个原子, 4 个化学键
坐标数据包含 5 个原子
SMILES键: 原子0(C) - 原子1(H): SINGLE
SMILES键: 原子0(C) - 原子2(H): SINGLE
SMILES键: 原子0(C) - 原子3(H): SINGLE
SMILES键: 原子0(C) - 原子4(H): SINGLE
匹配键: 原子0(C) - 原子1(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子2(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子3(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子4(H): 距离1.092Å, 类型SINGLE
非键连接: 原子1(H) - 原子2(H): 距离1.783Å
非键连接: 原子1(H) - 原子3(H): 距离1.783Å
非键连接: 原子1(H) - 原子4(H): 距离1.783Å
非键连接: 原子2(H) - 原子3(H): 距离1.783Å
非键连接: 原子2(H) - 原子4(H): 距离1.783Å
非键连接: 原子3(H) - 原子4(H): 距离1.783Å

=== 使用封装函数转换的图结构 ===
分子: C
节点数: 5
边数: 20
节点特征维度: 9
边特征维度: 6
分子属性: 15个量子化学属性


In [9]:
# 用所有 demo 数据测试封装函数
all_graphs = []
for smiles in extractor.get_smiles_list():
    mol_info = extractor.get_molecule_info(smiles)
    mol_data = {
        'smiles': mol_info['coordinates_data']['smiles'],
        'atoms': mol_info['coordinates_data']['atoms'],
        'coordinates': mol_info['coordinates_data']['coordinates'],
        'charges': mol_info['coordinates_data']['charges'],
        'properties': mol_info['properties_data']['properties']
    }
    graph = mol_to_graph(mol_data, distance_threshold)
    all_graphs.append(graph)
    print(f"转换分子 {smiles} 为图: {graph['num_nodes']} 个节点, {len(graph['edges'])} 条边")
print(f"\n总共转换了 {len(all_graphs)} 个分子图")


SMILES分子包含 5 个原子, 4 个化学键
坐标数据包含 5 个原子
SMILES键: 原子0(C) - 原子1(H): SINGLE
SMILES键: 原子0(C) - 原子2(H): SINGLE
SMILES键: 原子0(C) - 原子3(H): SINGLE
SMILES键: 原子0(C) - 原子4(H): SINGLE
匹配键: 原子0(C) - 原子1(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子2(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子3(H): 距离1.092Å, 类型SINGLE
匹配键: 原子0(C) - 原子4(H): 距离1.092Å, 类型SINGLE
非键连接: 原子1(H) - 原子2(H): 距离1.783Å
非键连接: 原子1(H) - 原子3(H): 距离1.783Å
非键连接: 原子1(H) - 原子4(H): 距离1.783Å
非键连接: 原子2(H) - 原子3(H): 距离1.783Å
非键连接: 原子2(H) - 原子4(H): 距离1.783Å
非键连接: 原子3(H) - 原子4(H): 距离1.783Å
转换分子 C 为图: 5 个节点, 20 条边
SMILES分子包含 4 个原子, 3 个化学键
坐标数据包含 4 个原子
SMILES键: 原子0(N) - 原子1(H): SINGLE
SMILES键: 原子0(N) - 原子2(H): SINGLE
SMILES键: 原子0(N) - 原子3(H): SINGLE
匹配键: 原子0(N) - 原子1(H): 距离1.017Å, 类型SINGLE
匹配键: 原子0(N) - 原子2(H): 距离1.017Å, 类型SINGLE
匹配键: 原子0(N) - 原子3(H): 距离1.017Å, 类型SINGLE
非键连接: 原子1(H) - 原子2(H): 距离1.619Å
非键连接: 原子1(H) - 原子3(H): 距离1.619Å
非键连接: 原子2(H) - 原子3(H): 距离1.619Å
转换分子 N 为图: 4 个节点, 12 条边
SMILES分子包含 3 个原子, 2 个化学键
坐标数据包含 3 个原子
SMILES键: 原子0(O) - 原子1(H): SINGLE

[16:30:30] Explicit valence for atom # 0 N, 4, is greater than permitted
[16:30:30] Explicit valence for atom # 2 N, 4, is greater than permitted
